In [125]:
import pandas as pd
import numpy as np

In [126]:
universe = pd.read_csv('../esg_nse_yahoo_final.csv')
universe.head()

,ticker
0,3MINDIA
1,AARTIDRUGS
2,AARTIIND
3,ABB
4,ACC


In [127]:
df = pd.read_json('../esg_data.json')
df.head()

,esg,esg_industry,industry,name,ticker
0,[],[],Mining (except Oil & Gas),20-Microns-Ltd,20MICRONS
1,"[[1622505600000, 14.44], [1625097600000, 14.46...","[[1622505600000, 13.83], [1625097600000, 12.9]...",Containers & Packaging Manufacturing,3M-INDIA-LIMITED,3MINDIA
2,"[[1633046400000, 36.21], [1635724800000, 35.68...","[[1633046400000, 19.94], [1635724800000, 20.25...",Brokerage & Capital Markets,5paisa-Capital-Limited,5PAISA
3,[],[],Brokerage & Capital Markets,A-K-CAPITAL-SERVICES-LIMITED,530499
4,[],[],"Architectural, Engineering, & Related Services",A2Z-MAINTENANCE-and-ENGINEERING-SERVICES-LIMITED,A2ZINFRA


In [128]:
# drop stocks not in universe
df = df[df['ticker'].isin(universe['ticker'])]

In [129]:
len(df)

383

In [130]:
df['ticker'].nunique() 
df[df['ticker'] == 'LT']

,esg,esg_industry,industry,name,ticker
725,"[[1619827200000, 89.93], [1622505600000, 93.2]...","[[1619827200000, 81.17], [1622505600000, 82.59...","Heavy & Civil Engineering Construction, Electr...",Larsen-and-Toubro,LT
831,[],[],"Architectural, Engineering, & Related Services",Nabha-Power-Limited,LT


In [131]:
df = df.drop(df[df['name'] == 'Nabha-Power-Limited'].index)

In [132]:
from datetime import datetime

def get_date(date: int) -> str:
    try:
        date = datetime.fromtimestamp(date/1000)
        return date.strftime("%Y-%m-%d")
    except TypeError:
        return str(date)

In [133]:
df = df.reset_index(drop=True)

for i in range(len(df)):
    esg = df.at[i, 'esg']
    esg_industry = df.at[i, 'esg_industry']
    
    for j in range(len(esg_industry)):
        esg_industry[j][0] = get_date(esg_industry[j][0])
    
    for j in range(len(esg)):
        esg[j][0] = get_date(esg[j][0])
    
    df.at[i, 'esg'] = esg
    df.at[i, 'esg_industry'] = esg_industry


In [134]:
df.tail()

,esg,esg_industry,industry,name,ticker
377,"[[2021-06-01, 75.33], [2021-07-01, 75.16], [20...","[[2021-06-01, 68.24], [2021-07-01, 67.15], [20...",Banking,THE-KARUR-VYSYA-BANK-LIMITED,KARURVYSYA
378,"[[2021-06-01, 43.94], [2021-07-01, 43.81], [20...","[[2021-06-01, 48.5], [2021-07-01, 48.18], [202...",Manufacturing,THE-SUPREME-INDUSTRIES-LIMITED,SUPREMEIND
379,"[[2023-01-01, 54.01], [2023-02-01, 47.85], [20...","[[2023-01-01, 57.93], [2023-02-01, 50.55], [20...",Construction Materials,The-Tinplate-Company-of-India-Limited,TINPLATE
380,"[[2023-05-01, 23.5]]","[[2023-05-01, 22.06]]",Paper Products,THE-WEST-COAST-PAPER-MILLS-LIMITED,WSTCSTPAPR
381,"[[2021-05-01, 94.81], [2021-06-01, 73.74], [20...","[[2021-05-01, 93.07], [2021-06-01, 70.21], [20...",Machinery Manufacturing,Thermax-Ltd,THERMAX


In [135]:
np.array(df.at[2, 'esg'])[:, 1].astype(float).mean()

53.71333333333333

In [136]:
def mean_esg(esg: list, ticker: str) -> float:
    try:
        esg = np.array(esg)
        yEsg = esg[:, 1].astype(float)
        return yEsg.mean()
    except (TypeError, ValueError, IndexError) as e:
        print(f"Error processing ESG data: {e} for ticker {ticker}")
        return np.nan  # Return a default value in case of error

def df_mean_esg(data: pd.DataFrame, col='esg', mean_func=mean_esg) -> float:
    mean_esgs = []
    for i in range(len(data)):
        try:
            mean_esgs.append(mean_func(data.at[i, col], df.at[i, 'ticker']))
        except KeyError:
            print(f"Column '{col}' not found in row {i}")
            mean_esgs.append(np.nan)  # Return a default value if the column doesn't exist
    return mean_esgs

In [137]:
df['mean_esg'] = df_mean_esg(df)
df['industry_mean_esg'] = df_mean_esg(df, col='esg_industry')
df.head()


,esg,esg_industry,industry,name,ticker,mean_esg,industry_mean_esg
0,"[[2021-06-01, 14.44], [2021-07-01, 14.46], [20...","[[2021-06-01, 13.83], [2021-07-01, 12.9], [202...",Containers & Packaging Manufacturing,3M-INDIA-LIMITED,3MINDIA,56.030000,48.452917
1,"[[2021-06-01, 16.98], [2021-07-01, 17.08], [20...","[[2021-06-01, 26.05], [2021-07-01, 26.42], [20...",Health Care & Pharmaceuticals,Aarti-Drugs-Ltd,AARTIDRUGS,21.845417,31.795833
2,"[[2021-06-01, 19.55], [2021-07-01, 19.69], [20...","[[2021-06-01, 28.24], [2021-07-01, 26.97], [20...","Chemicals, Plastics & Rubber Products Mfg.",AARTI-INDUSTRIES-LTD,AARTIIND,53.713333,55.423333
3,"[[2021-05-01, 98.83], [2021-06-01, 97.0], [202...","[[2021-05-01, 99.52], [2021-06-01, 98.11], [20...","Electronic Equipment & Instrumentation, Energy...",ABB-in-India,ABB,95.914800,96.465600
4,"[[2021-05-01, 75.23], [2021-06-01, 73.67], [20...","[[2021-05-01, 70.74], [2021-06-01, 79.18], [20...",Construction Materials,ACC-LIMITED,ACC,83.348800,86.453600


In [138]:
df['mean_esg'].isna().sum()

0

In [139]:
df['industry_mean_esg'].isna().sum()

0

In [140]:
df['industry_mean_esg'].describe()

count    382.000000
mean      55.037032
std       24.100547
min        0.836000
25%       36.382750
50%       56.093100
75%       73.768900
max      100.000000
Name: industry_mean_esg, dtype: float64

In [141]:
df['mean_esg'].describe()

count    382.000000
mean      56.479964
std       24.548262
min        1.576000
25%       35.636947
50%       58.332235
75%       76.914130
max       99.885000
Name: mean_esg, dtype: float64

In [142]:
(df['mean_esg'] > df['industry_mean_esg']).sum()


204

In [143]:
# Median of esg
def median_esg(esg: list, ticker: str) -> float:
    esg = np.array(esg)
    yEsg = esg[:, 1].astype(float)
    return np.median(yEsg)

df['median_esg'] = df_mean_esg(df, mean_func=median_esg)
df['industry_median_esg'] = df_mean_esg(df, col='esg_industry', mean_func=median_esg)
df.head()

,esg,esg_industry,industry,name,ticker,mean_esg,industry_mean_esg,median_esg,industry_median_esg
0,"[[2021-06-01, 14.44], [2021-07-01, 14.46], [20...","[[2021-06-01, 13.83], [2021-07-01, 12.9], [202...",Containers & Packaging Manufacturing,3M-INDIA-LIMITED,3MINDIA,56.030000,48.452917,61.660,52.050
1,"[[2021-06-01, 16.98], [2021-07-01, 17.08], [20...","[[2021-06-01, 26.05], [2021-07-01, 26.42], [20...",Health Care & Pharmaceuticals,Aarti-Drugs-Ltd,AARTIDRUGS,21.845417,31.795833,21.085,29.945
2,"[[2021-06-01, 19.55], [2021-07-01, 19.69], [20...","[[2021-06-01, 28.24], [2021-07-01, 26.97], [20...","Chemicals, Plastics & Rubber Products Mfg.",AARTI-INDUSTRIES-LTD,AARTIIND,53.713333,55.423333,57.645,60.130
3,"[[2021-05-01, 98.83], [2021-06-01, 97.0], [202...","[[2021-05-01, 99.52], [2021-06-01, 98.11], [20...","Electronic Equipment & Instrumentation, Energy...",ABB-in-India,ABB,95.914800,96.465600,96.510,96.880
4,"[[2021-05-01, 75.23], [2021-06-01, 73.67], [20...","[[2021-05-01, 70.74], [2021-06-01, 79.18], [20...",Construction Materials,ACC-LIMITED,ACC,83.348800,86.453600,86.700,88.840


In [144]:
df['median_esg'].describe()

count    382.000000
mean      57.616754
std       24.788685
min        1.650000
25%       39.271250
50%       60.740000
75%       77.813750
max       99.885000
Name: median_esg, dtype: float64

In [145]:
df['industry_median_esg'].describe()

count    382.000000
mean      56.108390
std       24.449886
min        0.880000
25%       39.397500
50%       58.240000
75%       74.347500
max      100.000000
Name: industry_median_esg, dtype: float64

In [146]:
((df['median_esg'] > df['industry_median_esg']).values & (df['mean_esg'] >  df['industry_mean_esg']).values).sum()

191

In [147]:
market_cap = pd.read_csv('../MCAP31032023_0.csv', names=['Sr. No.', 'Symbol', 'Company Name',
       'Market capitalization in Lakhs', 'CAP'], index_col=0)
market_cap.drop(['Sr. No.'], axis=0, inplace=True)
market_cap.head()

,Symbol,Company Name,Market capitalization in Lakhs,CAP
Sr. No.,,,,
1,RELIANCE,Reliance Industries Limited,"157,706,937.77",LARGECAP
2,TCS,Tata Consultancy Services Limited,"117,305,527.97",LARGECAP
3,HDFCBANK,HDFC Bank Limited,"89,808,750.01",LARGECAP
4,ICICIBANK,ICICI Bank Limited,"61,248,250.45",LARGECAP
5,HINDUNILVR,Hindustan Unilever Limited,"60,157,759.88",LARGECAP


In [148]:
market_cap['CAP'].unique()

array(['LARGECAP', 'MIDCAP', 'SMALLCAP', '-', nan], dtype=object)

In [149]:
df = df.merge(market_cap, how='left', left_on='ticker', right_on='Symbol')

In [150]:
df.head()

,esg,esg_industry,industry,name,ticker,mean_esg,industry_mean_esg,median_esg,industry_median_esg,Symbol,Company Name,Market capitalization in Lakhs,CAP
0,"[[2021-06-01, 14.44], [2021-07-01, 14.46], [20...","[[2021-06-01, 13.83], [2021-07-01, 12.9], [202...",Containers & Packaging Manufacturing,3M-INDIA-LIMITED,3MINDIA,56.030000,48.452917,61.660,52.050,3MINDIA,3M India Limited,"2,586,893.78",LARGECAP
1,"[[2021-06-01, 16.98], [2021-07-01, 17.08], [20...","[[2021-06-01, 26.05], [2021-07-01, 26.42], [20...",Health Care & Pharmaceuticals,Aarti-Drugs-Ltd,AARTIDRUGS,21.845417,31.795833,21.085,29.945,AARTIDRUGS,Aarti Drugs Limited,312757,SMALLCAP
2,"[[2021-06-01, 19.55], [2021-07-01, 19.69], [20...","[[2021-06-01, 28.24], [2021-07-01, 26.97], [20...","Chemicals, Plastics & Rubber Products Mfg.",AARTI-INDUSTRIES-LTD,AARTIIND,53.713333,55.423333,57.645,60.130,AARTIIND,Aarti Industries Limited,1878315,MIDCAP
3,"[[2021-05-01, 98.83], [2021-06-01, 97.0], [202...","[[2021-05-01, 99.52], [2021-06-01, 98.11], [20...","Electronic Equipment & Instrumentation, Energy...",ABB-in-India,ABB,95.914800,96.465600,96.510,96.880,ABB,ABB India Limited,"7,131,034.68",LARGECAP
4,"[[2021-05-01, 75.23], [2021-06-01, 73.67], [20...","[[2021-05-01, 70.74], [2021-06-01, 79.18], [20...",Construction Materials,ACC-LIMITED,ACC,83.348800,86.453600,86.700,88.840,ACC,ACC Limited,"3,130,789.25",LARGECAP


In [151]:
# Ensure you have a column named 'industry'. If not, you'll need to adjust the code or merge your dataframe with another one containing this information.

# Step 1: Select top 4 stocks from each industry based on mean_esg
top_stocks_by_industry = df.groupby('industry').apply(lambda x: x.nlargest(4, 'mean_esg')).reset_index(drop=True)

# Step 2: From this new universe, select the top stocks in 5:3:2 ratio of LARGECAP:MIDCAP:SMALLCAP
large_cap_stocks = top_stocks_by_industry[top_stocks_by_industry['CAP'] == 'LARGECAP'].nlargest(20, 'mean_esg')
mid_cap_stocks = top_stocks_by_industry[top_stocks_by_industry['CAP'] == 'MIDCAP'].nlargest(12, 'mean_esg')
small_cap_stocks = top_stocks_by_industry[top_stocks_by_industry['CAP'] == 'SMALLCAP'].nlargest(8, 'mean_esg')

# Combine all the selected stocks
selected_stocks = pd.concat([large_cap_stocks, mid_cap_stocks, small_cap_stocks]).reset_index(drop=True)


In [152]:
selected_stocks.head()

,esg,esg_industry,industry,name,ticker,mean_esg,industry_mean_esg,median_esg,industry_median_esg,Symbol,Company Name,Market capitalization in Lakhs,CAP
0,"[[2021-05-01, 99.43], [2021-06-01, 98.9], [202...","[[2021-05-01, 99.71], [2021-06-01, 99.77], [20...",Mining (except Oil & Gas),HINDUSTAN-ZINC-LTD,HINDZINC,99.2976,99.8084,99.58,100.00,HINDZINC,Hindustan Zinc Limited,"12,394,973.29",LARGECAP
1,"[[2021-05-01, 93.24], [2021-06-01, 95.96], [20...","[[2021-05-01, 93.24], [2021-06-01, 90.67], [20...",Commercial Banking,IndusInd-Bank-Limited,INDUSINDBK,97.9748,95.9528,98.53,97.22,INDUSINDBK,IndusInd Bank Limited,"8,285,787.88",LARGECAP
2,"[[2021-05-01, 98.4], [2021-06-01, 87.67], [202...","[[2021-05-01, 99.16], [2021-06-01, 90.52], [20...","Chemicals, Plastics & Rubber Products Mfg.",Tata-Chemicals-Ltd,TATACHEM,97.7936,98.7192,98.39,99.13,TATACHEM,Tata Chemicals Limited,"2,477,377.43",LARGECAP
3,"[[2021-05-01, 99.55], [2021-06-01, 88.27], [20...","[[2021-05-01, 99.01], [2021-06-01, 86.52], [20...",Machinery Manufacturing,Cummins-India-Ltd,CUMMINSIND,96.7960,97.0776,97.10,97.96,CUMMINSIND,Cummins India Limited,"4,517,251.20",LARGECAP
4,"[[2021-05-01, 92.88], [2021-06-01, 94.17], [20...","[[2021-05-01, 89.74], [2021-06-01, 90.17], [20...",Telecommunications,Tech-Mahindra-Ltd,TECHM,96.7716,95.8208,97.13,96.60,TECHM,Tech Mahindra Limited,"10,733,643.95",LARGECAP


In [153]:
pd.DataFrame(selected_stocks['ticker']).to_csv('selected_stocks.csv', index=False)

In [154]:
def compute_ndsv(esg_scores):
    n = len(esg_scores)
    ndsv = sum([(max(0, esg_scores[i - 1] - esg_scores[i]))**2 for i in range(1, n)]) / n
    return ndsv

In [155]:
df['ndsv'] = df['esg'].apply(lambda x: compute_ndsv(np.array(x)[:, 1].astype(float)))

In [156]:
df['ndsv'].describe()

count    382.000000
mean      16.689422
std       34.933605
min        0.000000
25%        1.286085
50%        5.513709
75%       17.820361
max      380.382970
Name: ndsv, dtype: float64

In [157]:
# Normalize the mean_esg and NDSV columns
df['normalized_mean_esg'] = (df['mean_esg'] - df['mean_esg'].mean()) / (df['mean_esg'].std())
df['normalized_NDSV'] = (df['ndsv'] - df['ndsv'].mean()) / (df['ndsv'].std())

# Compute the score for each stock
df['score'] = df['normalized_mean_esg'] - df['normalized_NDSV']

# Rank within each cap category and pick top stocks
top_large_cap = df[df['CAP'] == 'LARGECAP'].nlargest(20, 'score')
top_mid_cap = df[df['CAP'] == 'MIDCAP'].nlargest(12, 'score')
top_small_cap = df[df['CAP'] == 'SMALLCAP'].nlargest(8, 'score')

# Combine all the selected stocks
selected_stocks_ndsv = pd.concat([top_large_cap, top_mid_cap, top_small_cap]).reset_index(drop=True)


In [158]:
selected_stocks_ndsv.head()

,esg,esg_industry,industry,name,ticker,mean_esg,industry_mean_esg,median_esg,industry_median_esg,Symbol,Company Name,Market capitalization in Lakhs,CAP,ndsv,normalized_mean_esg,normalized_NDSV,score
0,"[[2021-05-01, 99.43], [2021-06-01, 98.9], [202...","[[2021-05-01, 99.71], [2021-06-01, 99.77], [20...",Mining (except Oil & Gas),HINDUSTAN-ZINC-LTD,HINDZINC,99.2976,99.8084,99.58,100.00,HINDZINC,Hindustan Zinc Limited,"12,394,973.29",LARGECAP,0.147752,1.744223,-0.473517,2.217740
1,"[[2021-05-01, 93.24], [2021-06-01, 95.96], [20...","[[2021-05-01, 93.24], [2021-06-01, 90.67], [20...",Commercial Banking,IndusInd-Bank-Limited,INDUSINDBK,97.9748,95.9528,98.53,97.22,INDUSINDBK,IndusInd Bank Limited,"8,285,787.88",LARGECAP,0.420616,1.690337,-0.465706,2.156044
2,"[[2021-05-01, 92.88], [2021-06-01, 94.17], [20...","[[2021-05-01, 89.74], [2021-06-01, 90.17], [20...",Telecommunications,Tech-Mahindra-Ltd,TECHM,96.7716,95.8208,97.13,96.60,TECHM,Tech Mahindra Limited,"10,733,643.95",LARGECAP,0.078464,1.641323,-0.475501,2.116824
3,"[[2021-05-01, 95.65], [2021-06-01, 96.59], [20...","[[2021-05-01, 94.55], [2021-06-01, 96.84], [20...",IT & Network Services,Tata-Consultancy-Services,TCS,96.3368,97.0920,96.79,97.77,TCS,Tata Consultancy Services Limited,"117,305,527.97",LARGECAP,0.449036,1.623611,-0.464893,2.088504
4,"[[2021-05-01, 99.83], [2021-06-01, 96.72], [20...","[[2021-05-01, 100.0], [2021-06-01, 96.24], [20...",Motor Vehicle Parts Manufacturing,Apollo-Tyres-Ltd,APOLLOTYRE,96.4412,96.1008,96.77,96.50,APOLLOTYRE,Apollo Tyres Limited,"2,031,687.93",LARGECAP,0.936176,1.627864,-0.450948,2.078812


In [159]:
len(selected_stocks_ndsv)

40

In [160]:
pd.DataFrame(selected_stocks_ndsv['ticker']).to_csv('selected_stocks_ndsv.csv', index=False)

In [161]:
print(set(selected_stocks_ndsv['ticker']) - set(selected_stocks['ticker']))

{'SUNCLAYLTD', 'DAAWAT', 'M&M', 'ELGIEQUIP'}
